In [70]:
import pandas as pd
import mysql.connector


cnx = mysql.connector.connect(
    host = "xxxxxxxx",
    user = "xxxxxxx",
    password = "xxxxxxx",
    database = "xxxxxx"
    )


if cnx.is_connected():
    print("Conectado ao Banco de dados")

itens_pedido = pd.read_sql_query("SELECT * FROM itens_pedido LIMIT 25057;", cnx)
pedido = pd.read_sql_query("SELECT * FROM pedido LIMIT 25056;", cnx)
produto = pd.read_sql_query("SELECT * FROM produto LIMIT 205;", cnx)
vendedores = pd.read_sql_query("SELECT * FROM vendedores LIMIT 1000;", cnx)


cnx.close()


Conectado ao Banco de dados


C:\Users\gabri\AppData\Local\Temp\ipykernel_8556\1828907175.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  itens_pedido = pd.read_sql_query("SELECT * FROM itens_pedido LIMIT 25057;", cnx)
C:\Users\gabri\AppData\Local\Temp\ipykernel_8556\1828907175.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pedido = pd.read_sql_query("SELECT * FROM pedido LIMIT 25056;", cnx)
C:\Users\gabri\AppData\Local\Temp\ipykernel_8556\1828907175.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  produto = pd.read_sql_query("SELECT * FR

In [71]:
df_list = {"itens_pedido":itens_pedido, "pedido":pedido, "produto":produto, "vendedores":vendedores}

for name, df in df_list.items():
    print(f"A quantidade de dados nulos na tabela {name} é {df.isna().sum().sum()}") 

A quantidade de dados nulos na tabela itens_pedido é 3
A quantidade de dados nulos na tabela pedido é 529
A quantidade de dados nulos na tabela produto é 0
A quantidade de dados nulos na tabela vendedores é 0


In [77]:
itens_pedido.isna().sum()

produto_id        0
pedido_id         1
quantidade        1
valor_unitario    0
valor_total       0
estado            0
frete             0
id_nf             1
dtype: int64

In [78]:
itens_pedido[itens_pedido.pedido_id.isna()]

,produto_id,pedido_id,quantidade,valor_unitario,valor_total,estado,frete,id_nf
25056,66,NaN,NaN,399.0,0.0,,0.0,NaN


Todos os valores nulos do dataframe itens_pedido estão em um único pedido, então irei remover esse registro do banco de dados.

In [80]:
pedido[pedido.vendedor_id.isna()]

,pedido_id,produto_id,vendedor_id,data_compra,total
5,23,21429,NaN,2020-12-16,2340.0
90,526,6888,NaN,2020-11-25,1400.0
100,592,91248,NaN,2020-06-26,180.0
230,1372,88368,NaN,2020-06-16,1000.0
317,1888,100983,NaN,2020-11-25,1750.0
...,...,...,...,...,...
24856,138786,24159,NaN,2020-07-16,810.0
24873,138847,9981,NaN,2020-11-25,4000.0
25003,139684,100484,NaN,2020-12-02,960.0
25045,139903,5995,NaN,2020-06-26,398.0


Já na tabela pedido, as coluna 'vendedor_id' possuí muitos dados nulos, pressuponho que esses pedidos tenham sido feitos pelo site, e por isso não houve um vendedor para intermediar, então irei adicionar a opção de "usuário" na lista de vendedores 

In [81]:
vendedores

,vendedor_id,nome_vendedor
0,1,Ana Duarte
1,2,Daniel Siqueira
2,3,Nádia Oliveira
3,4,Millena Pereira
4,5,Paulo Calanca
